# Build Model Matrices
This notebook implements code to build model matrices from design matrices. It uses a functional approach to enable flexibility in defining models.

## Packages

In [1]:
using Pkg
Pkg.add("Combinatorics")
using Combinatorics

using LinearAlgebra

    Updating registry at `C:\Users\benja\.julia\registries\General.toml`


   Resolving package versions...


  No Changes to `C:\Users\benja\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\benja\.julia\environments\v1.9\Manifest.toml`

## Initialization
For now we randomly initialization the design matrices.

In [2]:
# Fill an NxK matrix with values sampled from a uniform dist on [lower, upper]
function init(N, K, lower=-1, upper=1)
    lower .+ rand(N, K) .* (upper - lower)
end

init (generic function with 3 methods)

## Build Model Matrices
By defining a generic method that accepts a design matrix and a vector of functions, we can build model matrices by composing various transformations together.

In [40]:
function build_model_matrix(X::Array{Float64, 3}, funcs::Vector)
    g = (X) -> (i) -> X[:, :, i]
    hcat([func(g(X)) for func in funcs]...)
end

f = (x) -> x(1) .* x(2) .* x(3)

mat = rand(2,2,3)
build_model_matrix(mat, [f])

2×2 Matrix{Float64}:
 0.459343  0.00841885
 0.253677  0.844392

In [3]:
# Generic function for building model matrix
function build_model_matrix(X::Matrix{Float64}, funcs::Vector{Function})
    hcat([func(X) for func in funcs]...)
end

function build_intercept(X::Matrix{Float64})
    ones(size(X, 1), 1)
end

build_intercept (generic function with 1 method)

### Interaction Terms

In [4]:
# Build interaction terms for all combinations of columns length 2:order
# Returns a matrix with total columns sum([(K choose o) for o in 2:order])
# And rows equal to the number of rows in X
function build_interaction_terms(X::Matrix{Float64},; order::Int64=2)::Matrix{Float64}
    # K is the number of columns/factors in design matrix X
    K = size(X, 2)

    # Find interaction terms for each order and flatten
    interactions = vcat([collect(combinations(1:K, o)) for o in 2:order]...)

    # Build interaction terms
    return build_interaction_terms(X, interactions)
end

# Build interaction terms for a vector of interactions
function build_interaction_terms(X::Matrix{Float64}, interactions::Vector)::Matrix{Float64}
    # Get dimensions of design matrix
    N, K = size(X)

    # Initialize matrix to hold interaction terms
    M = zeros(N, length(interactions))

    # Compute interaction terms and fill matrix
    for (idx, combo) in enumerate(interactions)
        M[:, idx] = reduce(.*, [X[:, i] for i in combo])
    end

    return M
end

build_interaction_terms (generic function with 2 methods)

### Power Terms and Transformations

In [5]:
# Identity function for adding first order terms
function build_first_order_terms(X::Matrix{Float64})
    X
end

# Applies a function to a subset of the columns in X
# Returns a matrix with the same number of rows as X
# And the number of columns equal to the number of columns in X that satisfy the filter
function apply_function_to_filtered_columns(matrix::Matrix, func::Function, bool_filter::Vector{Bool})
    func.(matrix[:, bool_filter])
end

# Applies a function to a subset of the columns in X
# Each row of the filter corresponds with terms ordered 2:(# rows in filter)
# Each column corresponds with a column in X
function build_ordered_terms(X::Matrix{Float64}, order_filter::Matrix)::Matrix{Float64}
    order = size(order_filter, 1)
    order_filter = map(Bool, order_filter)
    f = (j) -> apply_function_to_filtered_columns(X, (x) -> x .^ j, order_filter[j, :])
    M = hcat(map(f, 2:order)...)
    return M
end

# Applies a function to a subset of the columns in X
# Each row of the filter corresponds with one of the functions
# Each column corresponds with a column in X
# The value of the i,jth entry indicates whether to apply the ith function to the jth column
function build_transform_terms(X::Matrix{Float64}, funcs::Vector{Function}, filter::Matrix)::Matrix{Float64}
    order_filter = map(Bool, filter)
    f = (j) -> apply_function_to_filtered_columns(X, funcs[j], order_filter[j, :])
    M = hcat(map(f, 1:length(funcs))...)
    return M
end

# Multiple dispatch version of build_transform_terms that takes a vector of functions
# func[i] is applied to column i of the design matrix
function build_transform_terms(X::Matrix{Float64}, funcs::Vector{Function})::Matrix{Float64}
    M = copy(X)
    for (f, col) in zip(funcs, eachcol(M))
        col .= f.(col)
    end

    return M
end

function build_ordered_terms(X::Matrix{Float64}, order::Int64=2)::Matrix{Float64}
    build_ordered_terms(X, map(Bool, ones(order, size(X, 2))))
end

build_ordered_terms (generic function with 3 methods)

### Basic API

In [12]:
# API function for easy model matrix construction
# Total columns is equal to 1 (intercept) + total power terms + total interaction terms + transformation terms
# Returns K by 1 + sum([2:o] for order in 2:order) + sum([(K choose o) for o in 2:order])   (disincludes first-order terms)
function build_model_matrix(X::Matrix{Float64},; order=0, interaction_order=0, transformations=[])
    funcs = [build_intercept, (x) -> build_interaction_terms(x, order=interaction_order)]

    if order > 1
        push!(funcs, (x) -> build_ordered_terms(x, order))
    end

    if length(transformations) > 0
        push!(funcs, (x) -> build_transform_terms(x, transformations))
    else
        push!(funcs, build_first_order_terms)
    end

    return build_model_matrix(X, funcs)
end    

build_model_matrix (generic function with 2 methods)

## Usage

In [24]:
# Initialize design matrix on 4 factors and 10 trials
D = init(10, 4)

# First order model
M0 = build_model_matrix(D)

# Fifth order model with fourth order interactions
M1 = build_model_matrix(D, order=5, interaction_order=4)

# Exp transform first factor; square transform second factor; first order term for third and fourth
# e.g. b_0 + b_1 * exp(x_1) + b_2 * x_2^2 + b_3 * x_3 + b_4 * x_4
funcs = [exp, x -> x .^ 2, x -> x, x -> x]
M2 = build_model_matrix(D, transformations=funcs)

10×5 Matrix{Float64}:
 1.0  1.02758   0.363127     0.721756  -0.657784
 1.0  1.11516   0.00404912   0.473507   0.000159593
 1.0  0.447006  0.117617    -0.322697  -0.606815
 1.0  1.12009   0.0281527   -0.490162   0.130619
 1.0  1.33806   0.501384     0.855255   0.291597
 1.0  1.934     0.0198699    0.247835  -0.440827
 1.0  1.33181   0.640902    -0.631984  -0.257078
 1.0  0.492865  0.41019      0.355557  -0.0248803
 1.0  0.724924  0.676462     0.84502    0.70577
 1.0  1.03442   0.197669    -0.771038   0.945153

## Filtering and Custom Transformations

In [14]:
# Custom interaction terms
test_mat = init(10, 3)
interactions = [[1, 2], [2, 3], [1, 2, 3]]
build_model_matrix(test_mat, [build_intercept, x -> build_interaction_terms(x, interactions)])

10×4 Matrix{Float64}:
 1.0  -0.0407838   0.000955586   0.000530795
 1.0   0.479819   -0.253261      0.218961
 1.0   0.0871234   0.25339       0.0405197
 1.0   0.071967    0.0202936     0.00209464
 1.0  -0.168897   -0.0422612    -0.0241276
 1.0  -0.0367064   0.123067      0.031328
 1.0   0.0256554  -0.0414339     0.0239416
 1.0  -0.131524    0.0305466    -0.00761354
 1.0   0.288168   -0.0976195     0.0535003
 1.0  -0.294756    0.127509     -0.118621

In [15]:
# Exponential transformation applied to first column and second column; square root applied to third; no transformation for fourth
filter = [1 1 0 0; 0 0 1 0; 0 0 0 1]
transforms = [exp, x -> sqrt(abs(x)), x -> x]
build_model_matrix(D, [build_intercept, x -> build_transform_terms(x, transforms, filter)])

10×5 Matrix{Float64}:
 1.0  0.626114  1.22425   0.696832  -0.223231
 1.0  1.02311   0.436306  0.13165    0.912343
 1.0  2.36738   0.657597  0.43004   -0.80163
 1.0  1.996     0.530053  0.404003   0.965241
 1.0  0.577019  0.911073  0.727836   0.418751
 1.0  0.72746   0.789927  0.80066   -0.347818
 1.0  0.877315  1.96439   0.888371   0.708003
 1.0  2.66176   1.86894   0.911345  -0.289429
 1.0  0.618767  0.762726  0.696583   0.453241
 1.0  0.437262  1.26595   0.897478   0.128568

## Examples

The following code block demonstrates the use of filters for applying power and interaction terms to create the model given by:

$$
    \beta_0 +\beta_1x_1 + \beta_2x_2 + \beta_3x_3 + \beta_4x_1x_2+\beta_5x_1x_3 + \beta_6x_2^2 + \beta_7x_3^2
$$

In [35]:
build_ordered_terms(D, power_filter)

10×2 Matrix{Float64}:
 0.422342    0.182263
 0.47756     0.795276
 0.394134    0.175238
 0.788176   -0.00276892
 0.513738   -0.108007
 0.37631    -0.278776
 0.139064   -0.275935
 0.138199    0.363694
 0.224344   -0.622429
 0.0895251  -0.000350206

In [34]:
# Initialize
D = init(10, 3)

# Interaction terms
interactions = [[1, 2], [1, 3]]

# Second order filter
# Apply second order terms to first and third columns
power_filter = [0 1 1; 1 0 0; 0 1 0]

# Define model
funcs = [build_intercept, build_first_order_terms, (X) -> build_ordered_terms(X, power_filter), (X) -> build_interaction_terms(X, interactions)]

# Build
M = build_model_matrix(D, funcs)

10×8 Matrix{Float64}:
 1.0  -0.649878   0.566978    0.67639   0.422342   …  -0.368467   -0.439571
 1.0  -0.691057   0.926487   -0.185307  0.47756       -0.640255    0.128057
 1.0  -0.627801   0.559598    0.5861    0.394134      -0.351316   -0.367954
 1.0   0.887793  -0.140422   -0.648464  0.788176      -0.124666   -0.575701
 1.0  -0.716755  -0.47623     0.330035  0.513738       0.34134    -0.236554
 1.0  -0.613441  -0.653259    0.89622   0.37631    …   0.400736   -0.549778
 1.0   0.372913  -0.651032   -0.999405  0.139064      -0.242778   -0.372691
 1.0   0.371752   0.713804   -0.126226  0.138199       0.265358   -0.0469246
 1.0   0.473649  -0.853814   -0.426554  0.224344      -0.404408   -0.202037
 1.0  -0.299208  -0.0704868  -0.714664  0.0895251      0.0210902   0.213833

Another example of an overly complicated model given by:

$$
    \beta_0+\beta_1\exp(x_1)+\beta_2x_2^2+\beta_3\exp(x_1)x_2^2x_3+\beta_4x_3+\beta_5x_1x_2x_3+\beta_5x_4^2
$$

In [21]:
# Define all transformations
exp_transf = (X) -> exp.(X[:, 1])
pow_transf = (X) -> X[:, 2] .^ 2
interaction_transf = (X) -> exp.(X[:, 1]) .* (X[:, 2] .^ 2) .* X[:, 3]
first_order_transf = (X) -> X[:, 3]
power_interaction_transf = (X) -> reduce(.*, [X[:, i] .^ i for i in 1:3])
third_order_transf = (X) -> X[:, 4] .^ 2

# Create vector of transformations
funcs = [build_intercept, exp_transf, pow_transf, interaction_transf, first_order_transf, power_interaction_transf, third_order_transf]

# Build model
build_model_matrix(D, funcs)

10×7 Matrix{Float64}:
 1.0  2.71295   0.0136396   -0.0211728   -0.572181  -0.00255005   0.112202
 1.0  1.86927   0.0130492   -0.0120041   -0.49212   -0.000972881  0.435646
 1.0  0.409109  0.0169838   -0.00522762  -0.752367   0.00646475   0.0527791
 1.0  2.44027   0.226294    -0.363771    -0.658745  -0.057709     0.054549
 1.0  0.615725  0.0691889    0.0172799    0.405619  -0.0022392    0.00499776
 1.0  0.911526  0.00674109   0.0027011    0.439584  -5.30434e-5   0.178403
 1.0  0.873933  0.996949     0.407638     0.467868  -0.0137588    0.30643
 1.0  1.41285   0.0142282    0.0142753    0.710132   0.00176096   0.726408
 1.0  2.44585   0.342844     0.243756     0.290689   0.00753201   9.53256e-5
 1.0  0.796256  0.979903     0.73178      0.937875  -0.184178     0.488806